# Multiquery + hybrid search + RAG-Fusion

In [11]:
import bs4
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

#### INDEXING ####

loader = WebBaseLoader(
    web_paths=("https://news.naver.com/section/101",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("sa_text", "sa_item _SECTION_HEADLINE")
        )
    ),
)
docs = loader.load()


# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300, 
    chunk_overlap=50)

# Make splits
splits = text_splitter.split_documents(docs)



In [12]:
import os
from dotenv import load_dotenv
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

if not openai_api_key:
    raise ValueError("openai api 키가 없습니다. 한번더 확인 부탁드립니다.")

os.environ['OPENAI_API_KEY'] = openai_api_key

In [13]:
# Index
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
vectorstore = Chroma.from_documents(documents=splits, 
                                    embedding=OpenAIEmbeddings())


In [ ]:
chroma_retriever = vectorstore.as_retriever(
    search_type="mmr", # MMR 알고리즘을 사용하여 검색
    search_kwargs={'k':1,'fetch_k':4} # 상위 1개의 문서를 반환하지만, 고려할 문서는 4개로 설정
)

In [14]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever
# Initialize the BM25 retriever
bm25_retriever = BM25Retriever.from_documents(splits)
bm25_retriever.k = 2  # Retrieve top 2 results

print("type of bm25", type(bm25_retriever))

type of bm25 <class 'langchain_community.retrievers.bm25.BM25Retriever'>


In [15]:
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, chroma_retriever], weights=[0.2, 0.8]
)

In [16]:
# 예시 고객 문의
query = "향후 집값에 대해서 알려줘"

# 관련 문서/제품 검색
docs = ensemble_retriever.get_relevant_documents(query)

# 각 문서에서 페이지 내용을 추출하여 출력
# for doc in docs:
#     print(doc.page_content)

# 검색된 문서들 출력
docs

[Document(metadata={'source': 'https://news.naver.com/section/101'}, page_content='맞벌이 부부인 장모(31)씨는 올해 초 서울 성동구 A아파트를 9억원에 매수했다. 집값의 70% 정도인 6억원을 30년 만기 고정금리형 주택담보대출과 신용 대출로 마련했다. 매달 원금과 이자를 합해 300만원을 빚 \n\n\n중앙일보\n\n1시간전'),
 Document(metadata={'source': 'https://news.naver.com/section/101'}, page_content='YTN\n\n59분전\n\n\n\n\n\n\n\n\n불붙은 집값에 컴백한 영끌족…주담대 증가의 40%는 2030'),
 Document(metadata={'source': 'https://news.naver.com/section/101'}, page_content='임종훈 한미사이언스 대표가 신동국 한양정밀 회장 등 3자 연합에 투자유치 방해 행위를 중단하라고 촉구했다. 특히 대규모 투자가 필요하다며 OCI홀딩스와 통합을 주장했던 송영숙 한미 회장과 임주현 부회장이 이제 와서 \n\n\n코메디닷컴\n\n53분전')]

In [17]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

template = """
당신은 AI 언어 모델 조수입니다. 당신의 임무는 주어진 사용자 질문에 대해 벡터 데이터베이스에서 관련 문서를 검색할 수 있도록 다섯 가지 다른 버전을 생성하는 것입니다. 
사용자 질문에 대한 여러 관점을 생성함으로써, 거리 기반 유사성 검색의 한계를 극복하는 데 도움을 주는 것이 목표입니다. 
각 질문은 새 줄로 구분하여 제공하세요. 원본 질문: {question}
"""
prompt_perspectives = ChatPromptTemplate.from_template(template)


generate_queries = (
    prompt_perspectives 
    | ChatOpenAI(model_name="gpt-4o-mini",temperature=0) 
    | StrOutputParser() 
    | (lambda x: x.split("\n"))
)

In [18]:
from langchain.load import dumps, loads

def reciprocal_rank_fusion(results: list[list], k=60, top_n=2):
    """ 
    여러 개의 순위가 매겨진 문서 리스트를 받아, RRF(Reciprocal Rank Fusion) 공식을 사용하여
    문서의 최종 순위를 계산하는 함수입니다. k는 RRF 공식에서 사용되는 선택적 파라미터이며,
    top_n은 반환할 우선순위가 높은 문서의 개수입니다.
    """
    
    # 각 고유한 문서에 대한 점수를 저장할 딕셔너리를 초기화합니다.
    fused_scores = {}

    # 순위가 매겨진 문서 리스트를 순회합니다.
    for docs in results:
        # 리스트 내의 각 문서와 그 문서의 순위를 가져옵니다.
        for rank, doc in enumerate(docs):
            # 문서를 문자열 형식으로 직렬화하여 딕셔너리의 키로 사용합니다 (문서가 JSON 형식으로 직렬화될 수 있다고 가정).
            doc_str = dumps(doc)
            # 해당 문서가 아직 딕셔너리에 없으면 초기 점수 0으로 추가합니다.
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            # 문서의 현재 점수를 가져옵니다 (이전에 계산된 점수).
            previous_score = fused_scores[doc_str]
            # RRF 공식을 사용하여 문서의 점수를 업데이트합니다: 1 / (순위 + k)
            fused_scores[doc_str] += 1 / (rank + k)

    # 문서들을 계산된 점수에 따라 내림차순으로 정렬하여 최종적으로 재정렬된 결과를 얻습니다.
    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]

    # 재정렬된 결과에서 우선순위가 높은 top_n 개의 문서만 반환합니다.
    return reranked_results[:top_n]

In [19]:

# RAG-Fusion 체인을 구성합니다.
# generate_queries: 질문에 대해 여러 검색 쿼리를 생성합니다.
# retriever.map(): 생성된 쿼리로 관련 문서들을 검색합니다.
# reciprocal_rank_fusion: 검색된 문서들을 RRF 알고리즘을 통해 결합하여 최종 순위를 계산합니다.
retrieval_chain_rag_fusion = generate_queries | ensemble_retriever.map() | reciprocal_rank_fusion

# 체인을 실행하여 질문에 대한 검색된 문서들을 가져옵니다.
question = "향후 집값에 대해서 알려줘"
docs = retrieval_chain_rag_fusion.invoke({"question": question})

# 검색된 고유 문서들의 개수를 출력합니다.
len(docs)

# 검색된 고유 문서들을 출력합니다.
docs

[(Document(metadata={'source': 'https://news.naver.com/section/101'}, page_content='올해 들어 서울 아파트값 상승세가 이어지는 가운데 26일 부동산R114가 올해 7∼8월 계약된 서울 아파트의 실거래가를 분석한 결과, 2021년 이후 동일 단지, 동일 주택형의 직전 최고가와 비교해 평균 90%까지 \n\n\n연합뉴스\n\n\n\n\n20\n개의 관련뉴스 더보기'),
  0.05),
 (Document(metadata={'source': 'https://news.naver.com/section/101'}, page_content='주택 후분양제 적용 확대가 필요하다는 주장이 제기됐다. 주택 건설 과정에서 일어날 수 있는 준공 지연과 같은 만약의 사태에서 수분양자(주택 소비자)를 최우선으로 보호할 수 있는 대책이라는 이유에서다. 서울주택도시공사\n\n\n조선비즈\n\n1시간전'),
  0.049189141547682)]

In [20]:
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough

# RAG
template = """다음 맥락을 바탕으로 질문에 답변하세요:

{context}

질문: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

final_rag_chain = (
    {"context": retrieval_chain_rag_fusion, 
     "question": RunnablePassthrough()} 
    | prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke(question)

'현재 서울 아파트값은 상승세를 보이고 있으며, 최근 분석에 따르면 2021년 이후 동일 단지, 동일 주택형의 직전 최고가와 비교해 평균 90%까지 상승한 것으로 나타났습니다. 이러한 추세가 지속된다면 향후 집값은 계속해서 오를 가능성이 있습니다. 또한, 주택 후분양제 적용 확대에 대한 논의가 이루어지고 있어, 이는 주택 소비자를 보호하는 데 기여할 수 있으며, 시장에 미치는 영향도 주목할 필요가 있습니다. 따라서 향후 집값은 여러 요인에 따라 변동할 수 있지만, 현재의 상승세가 계속될 가능성이 높습니다.'